In [1]:
from sys import modules

IN_COLAB = 'google.colab' in modules
if IN_COLAB:
    !pip install -q ir_axioms[examples] python-terrier

In [2]:
# Start/initialize PyTerrier.
from pyterrier import started, init

if not started():
    init(tqdm="auto")

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
from pyterrier.datasets import get_dataset, Dataset

# Load dataset.
dataset_name = "antique/test"
dataset: Dataset = get_dataset(f"irds:{dataset_name}")

In [4]:
from pathlib import Path
from pyterrier.index import IterDictIndexer
from ir_axioms.backend.pyterrier import IndexRef, IndexFactory

# Load documents and build index.
index_dir = Path(f"cache/indices/{dataset_name.replace('/', '-')}").absolute()
index_ref: IndexRef
if index_dir.exists():
    index_ref = IndexRef.of(str(index_dir))
else:
    # Don't forget to include the 'text' field in the meta index.
    indexer = IterDictIndexer(str(index_dir))
    index_ref = indexer.index(dataset.get_corpus_iter(), fields=["text"])
index = IndexFactory.of(index_ref)

In [5]:
# Load topics and qrels.
topics = dataset.get_topics()
qrels = dataset.get_qrels()

In [6]:
from pyterrier.batchretrieve import BatchRetrieve

# BM25 baseline retrieval, cut off at 10 documents per query.
pipeline_bm25 = BatchRetrieve(index, wmodel="BM25", num_results=10)
pipeline_bm25(topics)

,qid,docid,docno,rank,score,query
0,3990512,102622,3077638_1,0,15.887435,how can we get concentration onsomething
1,3990512,30676,3931664_0,1,15.621619,how can we get concentration onsomething
2,3990512,173781,4366141_0,2,15.395085,how can we get concentration onsomething
3,3990512,179429,1011598_10,3,15.134176,how can we get concentration onsomething
4,3990512,194913,4222212_0,4,15.134176,how can we get concentration onsomething
...,...,...,...,...,...,...
1947,1340574,87627,1972970_3,7,29.386170,why do some people only go to church on easter...
1948,1340574,357982,2980015_23,8,29.323234,why do some people only go to church on easter...
1949,1340574,116656,2036141_7,9,29.157311,why do some people only go to church on easter...
1950,1971899,234068,1971899_6,0,28.234530,what is masturbat


In [7]:
from ir_axioms.axiom import ORIG, REG_f, ANTI_REG_f, LEN_M_TDC, AND, LEN, ArgUC
from ir_axioms.backend.pyterrier.transformers import AxiomaticReranker

# Rerank the baseline retrieval using KwikSort with preferences from the axiom above.
pipeline_axiom = pipeline_bm25 >> AxiomaticReranker(
    (REG_f() & ANTI_REG_f() & LEN_M_TDC() & LEN(AND())) | ORIG(),
    index, dataset_name, verbose=True
)
pipeline_axiom(topics)

Reranking query axiomatically:   0%|          | 0/198 [00:00<?, ?query/s]

,docno,rank,score,qid,docid,query
0,3077638_1,1,10,3990512,102622,how can we get concentration onsomething
1,3931664_0,2,9,3990512,30676,how can we get concentration onsomething
2,4366141_0,3,8,3990512,173781,how can we get concentration onsomething
3,1011598_10,4,7,3990512,179429,how can we get concentration onsomething
4,4222212_0,5,6,3990512,194913,how can we get concentration onsomething
...,...,...,...,...,...,...
1947,1972970_3,8,3,1340574,87627,why do some people only go to church on easter...
1948,2980015_23,9,2,1340574,357982,why do some people only go to church on easter...
1949,2036141_7,10,1,1340574,116656,why do some people only go to church on easter...
1950,1971899_6,1,2,1971899,234068,what is masturbat


In [9]:
from ir_measures import nDCG, MAP
from pyterrier.pipelines import Experiment

# Compare the baseline retrieval with the reranked pipeline in an experiment.
results = Experiment(
    [pipeline_bm25, pipeline_axiom],
    topics,
    qrels,
    [nDCG @ 5, nDCG @ 10, nDCG @ 20, MAP],
    ["BM25", "BM25 + Axiomatic Reranking"],
)
results.sort_values("nDCG@10", ascending=False, inplace=True)
results

Reranking query axiomatically:   0%|          | 0/198 [00:00<?, ?query/s]

,name,nDCG@5,nDCG@10,nDCG@20,AP
1,BM25 + Axiomatic Reranking,0.529076,0.510466,0.381011,0.228424
0,BM25,0.529428,0.509908,0.380645,0.228586
